In [1]:
import numpy as np
from scipy.stats import levy_stable
from scipy.stats import uniform
from scipy.stats import multivariate_normal

In [2]:
def prior_sample(seed):
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    np.random.seed(seed)
    alpha = uniform.rvs(1.1, 2)  # U[1.1, 2]
    beta = uniform.rvs(-1, 1)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 300)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [3]:
def univ_alpha_stable_sampler(params,size,seed) :
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    #print(alpha,beta,gamma,delta)
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        #print(S,B)
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [4]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [5]:
def S(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return np.array([v_alpha, v_beta, v_gamma, mean_x])

In [6]:
def psi(y, x, covariance = np.diag([0.25,0.25,1,1])):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=covariance)


In [7]:
def pi_LF(theta,y) :
    """give the approximate density pi_LF

    Args:
        theta (_type_): _description_
        y (_type_): _description_
    """
    [K((summary_statistics(y)-summary_statistics(x_i))/epsilon)/epsilon for x_i in x]
    

In [8]:
def cov_estimate(theta_hat,n_draws = 1000):
    x = np.array([univ_alpha_stable_sampler(params = theta_hat, size = 200, seed=None) for _ in range(n_draws)])
    sumary_statistics = np.array([S(x_i) for x_i in x])

    return np.cov(sumary_statistics, rowvar=False)

In [9]:
def norm2(u) :
    return np.sqrt(np.sum(u*u))


In [10]:
def K(u, sigma) :
    d = sigma.shape[0]
    #print(d)
    return np.exp(-(u.T@np.linalg.inv(sigma)@u)/2)/(np.sqrt(2*np.pi)**d*np.sqrt(np.linalg.det(sigma)))




In [11]:
def resample(theta, weights,N) :
    normalized_weights = weigths/np.sum(weights)
    theta['t'] = np.random.choice(theta['t-1'], size=N, replace=True, p=normalized_weights)
    weights['t'] = np.ones(N)/N

In [12]:
def sample_from_M_t(N, theta_prev, weights_prev):

    samples = np.zeros((N, 4))
    normalized_weights = weights_prev/np.sum(weights_prev)
    for i in range(N):
        # Choose a component based on the weights
        component = np.random.choice(range(N), p=normalized_weights)
        # Generate a sample from the chosen component
        sample = np.random.multivariate_normal(mean = theta_prev[component], cov = np.diag([0.25,0.25,1,1]))
        samples[i] = sample
        
    return samples  

In [13]:
def M_t(N,theta_i,theta_prev, weights_prev) :
    densities = np.array([ psi(theta_i, theta_prev[i]) for i in range(N)])
    return np.sum(weights_prev*densities)
    

In [ ]:
def SMC_sampler(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    print('x.shape = ',x.shape)
    initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    print("weights = ", initial_weights)
    print(" sum(weights) = ", np.sum(initial_weights))
    weights = {"t" : initial_weights, "t-1" : initial_weights}
    t=1
    while t<T :
        epsilon = epsilons[t]
        i = 1
        while i<N : 
            # Mutation and correction : 
            theta_maj = theta['t']
            weights_maj = weights['t']
            theta['t'] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
            weights['t'] = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])/np.array([M_t(theta['t'][i],theta['t-1'],weights['t-1']) for i in range(N)])
            theta['t-1'] = theta_maj
            weights['t-1'] = weights_maj
            u = np.random.uniform(low=0, high=1, size=N)
            c_t = np.quantile(weights['t'], 0.9)
            proba = 1-np.minimum(1,weights['t']/c_t)
            rejected = theta['t'][u<=proba]
            accepted = theta['t'][u>proba]
            print("accepted = " ,accepted)
            print("acceptance pourcentage = ", len(accepted)*100/N)
            print("rejected = " ,rejected)
            print("rejected pourcentage = ", len(rejected)*100/N) 
            if proba<u : 
                continue
            else : 
                i+=1
                continue 
        # Resample : 
        resample(theta,weights,N)
        print("theta = ", theta)
        print("weights = ", weights)



    






In [21]:
def SMC_sampler_v2(arg, T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [3-0.01*k for k in range(301)]
    T = len(epsilons)

    #y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    #initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    #theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    #x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    #print('x.shape = ',x.shape)
    #initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    #print("weights = ", initial_weights)
    #print(" sum(weights) = ", np.sum(initial_weights))
    #weights = {"t" : initial_weights, "t-1" : initial_weights}
    y,x,theta,weights,sigma_hat = arg
    t=1
    res = []
    
    print("INITIALIZATION IS DONE :)")
    while t<T :
        print("t = ",t)
        epsilon = epsilons[t]
        print("epsilon = ",epsilon)
        accepted = []
        rejected = []
        while len(accepted)<N : 
            # Mutation and correction : 
            theta_maj = theta['t']
            weights_maj = weights['t']
            theta['t'] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
            print('theta t = ', theta['t'][:5])
            sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
            weights['t'] = np.array([K((S(y)-S(x[i]))/epsilon,sigma_hat)/epsilon for i in range(N)])/np.array([M_t(N,theta['t'][i],theta['t-1'],weights['t-1']) for i in range(N)])
            print('weights t = ', weights['t'][:5])
            theta['t-1'] = theta_maj
            weights['t-1'] = weights_maj
            u = np.random.uniform(low=0, high=1, size=N)
            c_t = np.quantile(weights['t'], 0.9)
            proba = 1-np.minimum(1,weights['t']/c_t)
            print('proba = ', proba[:10])
            rejected = theta['t'][u<=proba]
            accepted = theta['t'][u>proba]
            print("accepted = " ,accepted)
            print("acceptance pourcentage = ", len(accepted)*100/N)
            print("rejected = " ,rejected)
            print("rejected pourcentage = ", len(rejected)*100/N)
        t+=1
        res.append((theta['t'], weights['t']))
        # Resample : 
        resample(theta,weights,N)
        print("theta = ", theta)
        print("weights = ", weights)


In [22]:
def initialization(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [3-0.01*k for k in range(301)]
    T = len(epsilons)
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]

    sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
    print("sigma_hat = ", sigma_hat)
    print('det(sigma_hat) = ', np.linalg.det(sigma_hat))
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    print('x.shape = ',x.shape)
    initial_weights = np.array([K((S(y)-S(x[i]))/epsilon,sigma_hat)/epsilon for i in range(N)])
    print("weights = ", initial_weights)
    print(" sum(weights) = ", np.sum(initial_weights))
    weights = {"t" : initial_weights, "t-1" : initial_weights}

    return y,x,theta,weights,sigma_hat

In [23]:
arg = initialization()

sigma_hat =  [[ 2.46646825e+00 -1.15701962e-01 -9.34264769e-02 -6.40675752e+00]
 [-1.15701962e-01  2.20187667e-02 -2.56981993e-02  8.65411763e-01]
 [-9.34264769e-02 -2.56981993e-02  3.29325428e-01 -1.79375780e+00]
 [-6.40675752e+00  8.65411763e-01 -1.79375780e+00  3.37170252e+02]]
det(sigma_hat) =  3.302434718772186
x.shape =  (1000, 200)
weights =  [2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714617e-10 2.67714617e-10
 2.67714617e-10 2.67714617e-10 2.67714

In [24]:
SMC_sampler_v2(arg=arg)

INITIALIZATION IS DONE :)
t =  1
epsilon =  2.99


theta t =  [[   1.58748619    0.4252858   219.61032905 -119.47152066]
 [   1.89757822    0.24829282  219.9223489  -120.5325978 ]
 [   2.4515417    -0.64761593  217.80613744 -120.84922825]
 [   1.84295685   -0.49791288  220.22230236 -119.77410923]
 [   1.69453398    0.35262083  217.50816479 -118.63202322]]
weights t =  [0.00131146 0.00060246 0.0109999  0.00104627 0.02915365]
proba =  [0.93128041 0.9684314  0.42361478 0.94517634 0.         0.95434075
 0.95248925 0.74806918 0.29203646 0.86394269]
accepted =  [[ 1.69453398e+00  3.52620832e-01  2.17508165e+02 -1.18632023e+02]
 [ 1.69791812e+00 -5.15671081e-02  2.17561107e+02 -1.18832422e+02]
 [ 1.63310914e+00  8.80697810e-01  2.21873944e+02 -1.19996228e+02]
 ...
 [ 9.82345489e-01  6.14609600e-02  2.21340559e+02 -1.18347966e+02]
 [ 1.84057720e+00  2.03224851e-01  2.20554478e+02 -1.18077781e+02]
 [ 1.82913305e+00 -7.51405477e-01  2.19117535e+02 -1.17863073e+02]]
acceptance pourcentage =  27.6
rejected =  [[ 1.58748619e+00  4.25285800e-01  2.1

KeyboardInterrupt: 

In [ ]:
test = {'t' : [1,23,34]}
test['t'][1]

23

In [ ]:
[prior_sample(42) for _ in range(2)]

[[1.849080237694725,
  -0.049285693590083834,
  219.59818254342153,
  -120.40245474088903],
 [1.849080237694725,
  -0.049285693590083834,
  219.59818254342153,
  -120.40245474088903]]

In [ ]:
liste = np.array([prior_sample(42) for _ in range(2)])
liste

array([[ 1.84908024e+00, -4.92856936e-02,  2.19598183e+02,
        -1.20402455e+02],
       [ 1.84908024e+00, -4.92856936e-02,  2.19598183e+02,
        -1.20402455e+02]])

In [ ]:
liste.shape

(2, 4)